In [13]:
SUF = -3
sentences = [
    "judyta dała wczoraj stefanowi czekoladki",
    "babuleńka miała dwa rogate koziołki",
    "wczoraj wieczorem spotkałem pewną piękną kobietę",
    "dzisiaj rano zagrałem intensywny mecz w siatkówkę",
    "w naszej wannie chodzi wielka jaszczurka"
]

In [17]:
s_sets = [set(i.split()) for i in sentences]
s_sets_suffixes = [set([w[SUF:] for w in i.split()]) for i in sentences]
dictionary = {}
dictionary_suffixes = {}

with open('poleval_2grams.txt', 'r') as f:
    i = 0
    for line in f:
        i += 1
        if i%10000000 == 0:
            print(i)
        weight, root, subsequent = line.split()
        for s in range(len(s_sets)):
            if root in s_sets[s] and subsequent in s_sets[s]:
                if root not in dictionary:
                    dictionary[root] = {}
                dictionary[root][subsequent] = int(weight)
            if root[SUF:] in s_sets_suffixes[s] and subsequent[SUF:] in s_sets_suffixes[s]:
                if root[SUF:] not in dictionary_suffixes:
                    dictionary_suffixes[root[SUF:]] = {}
                dictionary_suffixes[root[SUF:]][subsequent[SUF:]] = int(weight)

10000000
20000000
30000000
40000000
50000000


In [18]:
with open('poleval_3grams.txt', 'r') as f:
    i = 0
    for line in f:
        i += 1
        if i%10000000 == 0:
            print(i)
        if len(line.split()) == 3:
            weight, root, subsequent1 = line.split()
            subsequent2 = None
            subsequent = subsequent1
            subsequent_s = subsequent1
        else:
            weight, root, subsequent1, subsequent2 = line.split()
            subsequent = ','.join([subsequent1,subsequent2])
            subsequent_s = ','.join([subsequent1[SUF:],subsequent2[SUF:]])
            
        for s in range(len(s_sets)):
            if root in s_sets[s] and subsequent1 in s_sets[s] and (subsequent2 == None or subsequent2 in s_sets[s]):
                if root not in dictionary:
                    dictionary[root] = {}
                dictionary[root][subsequent] = int(weight)
            if root[SUF:] in s_sets_suffixes[s] and subsequent1[SUF:] in s_sets_suffixes[s] and (subsequent2[SUF:] == None or subsequent2[SUF:] in s_sets_suffixes[s]):
                if root[SUF:] not in dictionary_suffixes:
                    dictionary_suffixes[root[SUF:]] = {}
                dictionary_suffixes[root[SUF:]][subsequent_s] = int(weight)

10000000
20000000
30000000
40000000
50000000
60000000
70000000
80000000
90000000
100000000
110000000
120000000
130000000
140000000
150000000
160000000
170000000


In [19]:
def rate(sentence):
    ans = 0
    for i in range(len(sentence) - 1):
        if sentence[i] in dictionary and sentence[i + 1] in dictionary[sentence[i]]:
            ans += dictionary[sentence[i]][sentence[i + 1]]
            if i < len(sentence) - 2:
                if ','.join([sentence[i + 1],sentence[i + 2]]) in dictionary[sentence[i]]:
                    ans += 5 * dictionary[sentence[i]][','.join([sentence[i + 1],sentence[i + 2]])]
    return ans
                

In [20]:
import itertools
import heapq
ans = []
for i in sentences:
    perm = list(itertools.permutations(i.split()))
    tmp_ans = heapq.nlargest(10,[[rate(p), p] for p in perm], key = lambda p: p[0])
    ans += tmp_ans

for a in ans:
    print(a[0], ' '.join(a[1]))

2 judyta dała wczoraj stefanowi czekoladki
2 judyta dała wczoraj czekoladki stefanowi
2 judyta stefanowi dała wczoraj czekoladki
2 judyta stefanowi czekoladki dała wczoraj
2 judyta czekoladki dała wczoraj stefanowi
2 judyta czekoladki stefanowi dała wczoraj
2 dała wczoraj judyta stefanowi czekoladki
2 dała wczoraj judyta czekoladki stefanowi
2 dała wczoraj stefanowi judyta czekoladki
2 dała wczoraj stefanowi czekoladki judyta
184 babuleńka miała dwa koziołki rogate
184 babuleńka rogate miała dwa koziołki
184 miała dwa koziołki babuleńka rogate
184 miała dwa koziołki rogate babuleńka
184 rogate babuleńka miała dwa koziołki
184 rogate miała dwa koziołki babuleńka
183 babuleńka miała dwa rogate koziołki
183 babuleńka koziołki miała dwa rogate
183 miała dwa rogate babuleńka koziołki
183 miała dwa rogate koziołki babuleńka
459 spotkałem wczoraj wieczorem pewną piękną kobietę
459 pewną piękną kobietę spotkałem wczoraj wieczorem
458 wczoraj wieczorem spotkałem pewną piękną kobietę
458 spotkał

In [37]:
def rate_better(sentence):
    ans = 0
    for i in range(len(sentence) - 1):
        if sentence[i] in dictionary and sentence[i + 1] in dictionary[sentence[i]]:
            ans += dictionary[sentence[i]][sentence[i + 1]]
            if i < len(sentence) - 2:
                if ','.join([sentence[i + 1],sentence[i + 2]]) in dictionary[sentence[i]]:
                    ans += 2 * dictionary[sentence[i]][','.join([sentence[i + 1],sentence[i + 2]])]
        if sentence[i][SUF:] in dictionary_suffixes and sentence[i + 1][SUF:] in dictionary_suffixes[sentence[i][SUF:]]:
            ans += dictionary_suffixes[sentence[i][SUF:]][sentence[i + 1][SUF:]]
            if i < len(sentence) - 2:
                if ','.join([sentence[i + 1][SUF:],sentence[i + 2][SUF:]]) in dictionary_suffixes[sentence[i][SUF:]]:
                    ans += 2 * dictionary_suffixes[sentence[i][SUF:]][','.join([sentence[i + 1][SUF:],sentence[i + 2][SUF:]])]
    return ans

In [36]:
ans = []
for i in sentences:
    perm = list(itertools.permutations(i.split()))
    tmp_ans = heapq.nlargest(10,[[rate_better(p), p] for p in perm], key = lambda p: p[0])
    ans += tmp_ans

for a in ans:
    print(a[0], ' '.join(a[1]))

63 stefanowi wczoraj dała czekoladki judyta
63 czekoladki judyta stefanowi wczoraj dała
61 stefanowi wczoraj czekoladki dała judyta
61 czekoladki dała judyta stefanowi wczoraj
58 judyta stefanowi wczoraj dała czekoladki
58 judyta czekoladki stefanowi wczoraj dała
58 stefanowi wczoraj dała judyta czekoladki
58 czekoladki stefanowi wczoraj dała judyta
55 judyta stefanowi wczoraj czekoladki dała
55 judyta czekoladki dała stefanowi wczoraj
584 babuleńka miała dwa rogate koziołki
584 babuleńka miała dwa koziołki rogate
584 babuleńka rogate miała dwa koziołki
579 babuleńka koziołki miała dwa rogate
579 miała dwa rogate koziołki babuleńka
579 miała dwa koziołki babuleńka rogate
579 miała dwa koziołki rogate babuleńka
579 rogate babuleńka miała dwa koziołki
579 rogate miała dwa koziołki babuleńka
579 koziołki babuleńka miała dwa rogate
1708 pewną piękną kobietę spotkałem wczoraj wieczorem
1706 spotkałem wczoraj wieczorem pewną piękną kobietę
1705 wczoraj wieczorem spotkałem pewną piękną kobiet